## <a id='toc1_1_'></a>[Quantum Machine Learning](#toc0_)

**Table of contents**<a id='toc0_'></a>    
- [Quantum Machine Learning](#toc1_1_)    
  - [Environment Setup](#toc1_2_)    
  - [About Dataset](#toc1_3_)    
  - [Load train and test data](#toc1_4_)    
  - [Initialize the weights and bias for the training](#toc1_5_)    
  - [Load Qunatum Machine Learning Package](#toc1_6_)    
  - [Train Model](#toc1_7_)    
  - [Validate model](#toc1_8_)    
  - [Classify](#toc1_9_)    
  - [Conclusion](#toc1_10_)    
  - [References](#toc1_11_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## <a id='toc1_2_'></a>[Environment Setup](#toc0_)

- Executed in docker container built using the Dockerfile in the QuantumKatas repository.
- It had few issues with iqsharp and proj deps. Fixed it. See [PR](https://github.com/microsoft/QuantumKatas/pull/900)

## <a id='toc1_3_'></a>[About Dataset](#toc0_)

The task is to implement a quantum machine learning algorithm for banknote authentication. The dataset is available at [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/banknote+authentication).  

The dataset contains 1372 data points, each with 4 features and a label. The task is to classify the data points into two classes: genuine and forged banknotes.

## <a id='toc1_4_'></a>[Load train and test data](#toc0_)

In [1]:
import json

# load data from json file
with open('train_test_split.json', 'r') as f:
    data = json.load(f)
    
X_train_list = data['train']['features']
y_train_list = data['train']['labels']

X_test_list = data['test']['features']
y_test_list = data['test']['labels']

## <a id='toc1_5_'></a>[Initialize the weights and bias for the training](#toc0_)

This following block of code initializes the weights and bias for the training randomly with uniform distribution.

In [2]:
## Generate Initial Weights

import random
# Set a seed for reproducibility
random.seed(42)

# Function to generate random numbers between 0 and 5
def generate_random_numbers():
    return random.uniform(0, 5)


initial_weights = []
for i in range(8):
    arr = []
    for i in range(8):
        arr.append(generate_random_numbers())
    initial_weights.append(arr)
    
initial_weights

[[3.1971339922894186,
  0.12505377611333468,
  1.3751465918455963,
  1.1160536907441139,
  3.682356070820062,
  3.3834974371145563,
  4.460897838524227,
  0.43469416314708076],
 [2.1096090984263522,
  0.14898609719035172,
  1.0931898740180168,
  2.526776440516812,
  0.13267984841931812,
  0.9941882534332425,
  3.249422188897616,
  2.724707403016083],
 [1.1022031102034835,
  2.946328419379544,
  4.047152283389133,
  0.032493798390305084,
  4.02909625916404,
  3.4906969749411343,
  1.7012525825899594,
  0.7773974990589078],
 [4.786065361033906,
  1.682972725563134,
  0.46372921690073954,
  0.48358188416732006,
  4.237471831737299,
  3.0186301568344556,
  4.035641366371901,
  3.6486589334690893],
 [2.6811404572735036,
  4.865578819896853,
  1.8926718860417675,
  2.760203156366135,
  4.1470233212649745,
  3.0925987618212303,
  4.308534501553886,
  2.88676072628381],
 [3.5228591810746175,
  0.22912191827831108,
  1.1394913782577343,
  1.4469398180105357,
  0.3989598846181375,
  1.1639544318

## <a id='toc1_6_'></a>[Load Qunatum Machine Learning Package](#toc0_)

In [3]:
import qsharp
qsharp.packages.add("Microsoft.Quantum.MachineLearning::0.28.302812")


Preparing Q# environment...


Adding package Microsoft.Quantum.MachineLearning::0.28.302812.

In [4]:
qsharp.reload()


Reloading workspace.

## <a id='toc1_7_'></a>[Train Model](#toc0_)

In [5]:
from Microsoft.Quantum.Samples import (
    TrainModel, ValidateModel, ClassifyModel
)

In [8]:
weights, bias = TrainModel.simulate(
    trainingVectors=X_train_list,
    trainingLabels=y_train_list,
    initialParameters=initial_weights,
    verbose=True
)

Ready to train.
Classifier structure length: 4
Mapped length: 8
Example Map - Mapped[0]: SequentialModel(([ControlledRotation(((0, []), PauliX, 4)),ControlledRotation(((0, []), PauliZ, 5)),ControlledRotation(((1, []), PauliX, 1)),ControlledRotation(((1, []), PauliX, 0))], [3.1971339922894186,0.12505377611333468,1.3751465918455963,1.1160536907441139,3.682356070820062,3.3834974371145563,4.460897838524227,0.43469416314708076], 0))
Samples length: 1097
Mapped length: 8
  Beginning training at start point #0...
    Pre-encoding samples...
    Beginning epoch 1.
        Beginning minibatch 0 of 1.
      Estimating gradient at sample 0...
      Estimating gradient at sample 1...
      Estimating gradient at sample 2...
      Estimating gradient at sample 3...
      Estimating gradient at sample 4...
      Estimating gradient at sample 5...
      Estimating gradient at sample 6...
      Estimating gradient at sample 7...
      Estimating gradient at sample 8...
      Estimating gradient at sam

## <a id='toc1_8_'></a>[Validate model](#toc0_)

In [9]:

miss_rate = ValidateModel.simulate(
    validationVectors=X_test_list,
    validationLabels=y_test_list,
    parameters=weights, bias=bias
)

print(f"Miss rate: {miss_rate}")

Miss rate: 0.1454545454545


## <a id='toc1_9_'></a>[Classify](#toc0_)

In [10]:
actual_labels = y_test_list
classified_labels = ClassifyModel.simulate(
    samples=X_test_list,
    parameters=weights, bias=bias,
    tolerance=0.005, nMeasurements=10_000
)

import pandas as pd
df = pd.DataFrame({'actual': actual_labels, 'classified': classified_labels})
df = df[df['actual'] != df['classified']]

df

,actual,classified
11,1,0
13,1,0
17,1,0
19,1,0
28,0,1
38,1,0
41,1,0
44,1,0
72,1,0
77,1,0


## <a id='toc1_10_'></a>[Conclusion](#toc0_)



Accuracy of the classical model:
    
    Accuracy: 98%
    Time taken to train: 10.1 ms
        
Accuracy of using Quantum model:
    
    Accuracy: 85.54%
    Num Qubits Used: 3
    Time taken to train: ~12Hours
        
    

## <a id='toc1_11_'></a>[References](#toc0_)

- The paper ['Circuit-centric quantum classifiers', by Maria Schuld, Alex Bocharov, Krysta Svore and Nathan Wiebe](https://arxiv.org/abs/1804.00633). Used it to understand the concept of quantum classifiers and the implementation of the quantum circuit for the same.
- [QunatumKatas/tutorials/QuantumClassification](https://github.com/microsoft/QuantumKatas/tree/main/tutorials/QuantumClassification) provided the basic structure of the code and the implementation of the quantum circuit for the quantum classifier.

- [Quantum Machine Learning Samples](https://github.com/microsoft/Quantum/tree/main/samples/machine-learning) from Microsoft Quantum repository.